In [2]:
import json
import cv2
from glob import glob
from sklearn.model_selection import train_test_split

In [3]:
!pwd

/home/jupyter


In [5]:
!ls

DETR.ipynb	    Preparation.ipynb  open.zip		      snap  tutorials
LabelMe2Coco.ipynb  dacon	       sample_submission.csv  src


In [7]:
%cd dacon/data

/home/jupyter/dacon/data


In [3]:
train_path = '/home/ubuntu/workspace/datasets/dacon/labelme/*.txt'
ann_train = glob(train_path)
val_path = '/home/ubuntu/workspace/datasets/dacon/labelme/*.txt'
ann_valid = glob(val_path)
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", "chevrolet_spark_hatchback_2016_2021", 
           "chevrolet_trailblazer_suv_2021_", "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", "genesis_g80_sedan_2021_", 
           "genesis_gv80_suv_2020_", "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", "hyundai_grandeur_sedan_2011_2016", 
           "hyundai_grandstarex_van_2018_2020", "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
           "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
           "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", "kia_morning_hatchback_2004_2010", 
           "kia_morning_hatchback_2011_2016", "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", "kia_sorrento_suv_2020_", 
           "kia_soul_suv_2014_2018", "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_",
           "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]

In [4]:
def labelmetxt2coco(classes, target_file_path, anns):
    obj = {}
    # 숫자로 해도 되지만, 클래스 확인하기 위해 차종으로 변경
    obj["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
    obj["images"] = []
    obj["annotations"] = []
    cnt_ann = 0
    for i, ann in enumerate(anns):
        img_path = ann.replace("txt", "png")
        img_path = img_path.replace("label","train")
        split_words = img_path.split('/')
        img_path = split_words[-1]
        #h, w, _ = cv2.imread(img_path).shape
        obj["images"].append({"id":i,"height":1040,"width":1920,"file_name":img_path})

        f_ann = open(ann, "r")
        for line in f_ann.readlines():
            data = line.split()
            cat = int(float(data[0]))
            pt1x = int(data[1])
            pt1y = int(data[2])
            #pt2x = int(data[3])    not to use
            #pt2y = int(data[4])
            pt3x = int(data[5])
            pt3y = int(data[6])
            #pt4x = int(data[7])
            #pt4y = int(data[8])
            x = pt1x
            y = pt1y
            width = pt3x - pt1x
            height = pt3y - pt1y
            area = width * height
            obj["annotations"].append({"id": cnt_ann,
                                        "image_id": i,
                                        "category_id": cat,
                                        "bbox": [x, y, width, height],
                                        "area": area,
                                        "segmentation": [],
                                        "iscrowd": 0})
            cnt_ann += 1
        
    with open(target_file_path, "w") as f:
        json.dump(obj, f, ensure_ascii=False, indent=4)

In [6]:
path = '/home/ubuntu/workspace/datasets/dacon/testing_transfers/coco/annotations'
file_name = 'train.json'
target_file_path = path + '/' + file_name

labelmetxt2coco(classes, target_file_path, ann_train)